### Model to predict number of available bikes

In [57]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
pd.options.mode.chained_assignment=None
%matplotlib inline
import os
import pymysql
import pandas as pd

host = 'dublinbikes.cb2pu3bkmmlf.us-east-2.rds.amazonaws.com'
user = 'master'
password = 'master-50'
db = 'dublinbikes'
try:
    con = pymysql.connect(host=host,user=user,password=password,db=db, use_unicode=True, charset='utf8')
    print('+=========================+')
    print('|  CONNECTED TO DATABASE  |')
    print('+=========================+')
    
except Exception as e:
        sys.exit('error',e)



#cur = con.cursor()
#cur.execute("SELECT * FROM dublinbikes.availability")
#data = cur.fetchall()
df = pd.read_sql_query("SELECT * FROM dublinbikes.availability",con)
df.head(10)



+=========================+
|  CONNECTED TO DATABASE  |
+=========================+


,ID,number,available_bikes,available_bike_stands,last_update,day_of_week,hour_d,mins,date_of_fetching
0,1,42,12,18,2147483647,Friday,22,13,2019-02-22
1,2,30,3,17,2147483647,Friday,22,13,2019-02-22
2,3,54,4,29,2147483647,Friday,22,13,2019-02-22
3,4,108,11,28,2147483647,Friday,22,13,2019-02-22
4,5,56,2,38,2147483647,Friday,22,13,2019-02-22
5,6,6,5,14,2147483647,Friday,22,13,2019-02-22
6,7,18,14,16,2147483647,Friday,22,13,2019-02-22
7,8,32,24,6,2147483647,Friday,22,13,2019-02-22
8,9,52,0,32,2147483647,Friday,22,13,2019-02-22
9,10,48,3,37,2147483647,Friday,22,13,2019-02-22


In [58]:
df.shape

df.head(329)

,ID,number,available_bikes,available_bike_stands,last_update,day_of_week,hour_d,mins,date_of_fetching
0,1,42,12,18,2147483647,Friday,22,13,2019-02-22
1,2,30,3,17,2147483647,Friday,22,13,2019-02-22
2,3,54,4,29,2147483647,Friday,22,13,2019-02-22
3,4,108,11,28,2147483647,Friday,22,13,2019-02-22
4,5,56,2,38,2147483647,Friday,22,13,2019-02-22
5,6,6,5,14,2147483647,Friday,22,13,2019-02-22
6,7,18,14,16,2147483647,Friday,22,13,2019-02-22
7,8,32,24,6,2147483647,Friday,22,13,2019-02-22
8,9,52,0,32,2147483647,Friday,22,13,2019-02-22
9,10,48,3,37,2147483647,Friday,22,13,2019-02-22


In [59]:
df_2 = pd.read_sql_query("SELECT * FROM dublinbikes.weather",con)


In [60]:
df_2.head(50)

,ID,temperature,day_of_week,wind_speed,cloud_coverage,hour_d,mins,date_of_fetching
0,1,12.58,Friday,4.6,Clouds,22,13,2019-02-22
1,2,11.51,Friday,7.7,Clouds,22,28,2019-02-22
2,3,11.51,Friday,7.7,Clouds,22,43,2019-02-22
3,4,11.51,Friday,7.7,Clouds,22,58,2019-02-22
4,5,11.58,Friday,7.2,Clouds,23,13,2019-02-22
5,6,11.59,Friday,7.2,Clouds,23,28,2019-02-22
6,7,11.51,Friday,7.7,Clouds,23,43,2019-02-22
7,8,11.57,Friday,7.7,Clouds,23,58,2019-02-22
8,9,11.57,Saturday,7.7,Clouds,0,13,2019-02-23
9,10,11.58,Saturday,7.7,Clouds,0,28,2019-02-23


In [61]:
df_2.shape

(4377, 8)

In [62]:
#df.describe().T

In [63]:
print("Unique values for:\n- mins in weather:", pd.unique(df_2.mins.ravel()))

Unique values for:
- mins in weather: [13 28 43 58 14 29 44 59 30 45  0 15 31 46  1 16 32 47  2 17  3 18 33 48
 19 34 49  4  5 20 35 50 21 36 51  6  7 22 37 52 23 38 53  8 24 39 54  9
 10 25 40 55 11 26 41 56 57 12 27 42]


In [64]:
print("Unique values for:\n- mins in availability:", pd.unique(df.mins.ravel()))

Unique values for:
- mins in availability: [13 21 28 35 42 49 56  3 10 18 25 32 39 46 53  0  8 15 22 29 36 43 50 57
  5 12 19 26 33 40 48 55  2  9 16 23 30 38 45 52 59  6 20 17  7 47 37 27
  4 11  1 51 58 41 24 31 14 54 44 34]


In [65]:
#set(df['mins']).intersection(set(df_2['mins']))

In [66]:
#set(df['mins']).union(set(df_2['mins']))

In [67]:
mergedStuff = pd.merge(df, df_2, on=['mins','date_of_fetching','hour_d'], how='inner')
mergedStuff.head(226)

,ID_x,number,available_bikes,available_bike_stands,last_update,day_of_week_x,hour_d,mins,date_of_fetching,ID_y,temperature,day_of_week_y,wind_speed,cloud_coverage
0,1,42,12,18,2147483647,Friday,22,13,2019-02-22,1,12.58,Friday,4.6,Clouds
1,2,30,3,17,2147483647,Friday,22,13,2019-02-22,1,12.58,Friday,4.6,Clouds
2,3,54,4,29,2147483647,Friday,22,13,2019-02-22,1,12.58,Friday,4.6,Clouds
3,4,108,11,28,2147483647,Friday,22,13,2019-02-22,1,12.58,Friday,4.6,Clouds
4,5,56,2,38,2147483647,Friday,22,13,2019-02-22,1,12.58,Friday,4.6,Clouds
5,6,6,5,14,2147483647,Friday,22,13,2019-02-22,1,12.58,Friday,4.6,Clouds
6,7,18,14,16,2147483647,Friday,22,13,2019-02-22,1,12.58,Friday,4.6,Clouds
7,8,32,24,6,2147483647,Friday,22,13,2019-02-22,1,12.58,Friday,4.6,Clouds
8,9,52,0,32,2147483647,Friday,22,13,2019-02-22,1,12.58,Friday,4.6,Clouds
9,10,48,3,37,2147483647,Friday,22,13,2019-02-22,1,12.58,Friday,4.6,Clouds


In [68]:
mergedStuff.tail(226)

,ID_x,number,available_bikes,available_bike_stands,last_update,day_of_week_x,hour_d,mins,date_of_fetching,ID_y,temperature,day_of_week_y,wind_speed,cloud_coverage
39324,522626,42,0,30,2147483647,Friday,6,12,2019-04-05,4360,0.27,Friday,3.1,Clouds
39325,522627,30,5,15,2147483647,Friday,6,12,2019-04-05,4360,0.27,Friday,3.1,Clouds
39326,522628,54,3,30,2147483647,Friday,6,12,2019-04-05,4360,0.27,Friday,3.1,Clouds
39327,522629,108,0,40,2147483647,Friday,6,12,2019-04-05,4360,0.27,Friday,3.1,Clouds
39328,522630,56,0,40,2147483647,Friday,6,12,2019-04-05,4360,0.27,Friday,3.1,Clouds
39329,522631,6,0,20,2147483647,Friday,6,12,2019-04-05,4360,0.27,Friday,3.1,Clouds
39330,522632,18,24,6,2147483647,Friday,6,12,2019-04-05,4360,0.27,Friday,3.1,Clouds
39331,522633,32,22,8,2147483647,Friday,6,12,2019-04-05,4360,0.27,Friday,3.1,Clouds
39332,522634,52,16,16,2147483647,Friday,6,12,2019-04-05,4360,0.27,Friday,3.1,Clouds
39333,522635,48,15,25,2147483647,Friday,6,12,2019-04-05,4360,0.27,Friday,3.1,Clouds


In [69]:
mergedStuff.shape

(39550, 14)

In [70]:
print("Unique values for:\n- clouds in weather:", pd.unique(df_2.cloud_coverage.ravel()))

Unique values for:
- clouds in weather: ['Clouds' 'Clear' 'Rain' 'Fog' 'Drizzle' 'Mist' 'Snow']


In [71]:
#data_input1=pd.DataFrame(Train_data['Sex'])
data_input1= pd.DataFrame(mergedStuff['day_of_week_x'])
data_input2= pd.DataFrame(mergedStuff['cloud_coverage'])
dummy=pd.get_dummies(data_input1)
dummy_2=pd.get_dummies(data_input2)
mergedStuff=pd.concat([mergedStuff,dummy],axis=1)
mergedStuff=pd.concat([mergedStuff,dummy_2],axis=1)








In [72]:
mergedStuff.tail(226)

,ID_x,number,available_bikes,available_bike_stands,last_update,day_of_week_x,hour_d,mins,date_of_fetching,ID_y,...,day_of_week_x_Sunday,day_of_week_x_Thursday,day_of_week_x_Tuesday,day_of_week_x_Wednesday,cloud_coverage_Clear,cloud_coverage_Clouds,cloud_coverage_Drizzle,cloud_coverage_Fog,cloud_coverage_Mist,cloud_coverage_Rain
39324,522626,42,0,30,2147483647,Friday,6,12,2019-04-05,4360,...,0,0,0,0,0,1,0,0,0,0
39325,522627,30,5,15,2147483647,Friday,6,12,2019-04-05,4360,...,0,0,0,0,0,1,0,0,0,0
39326,522628,54,3,30,2147483647,Friday,6,12,2019-04-05,4360,...,0,0,0,0,0,1,0,0,0,0
39327,522629,108,0,40,2147483647,Friday,6,12,2019-04-05,4360,...,0,0,0,0,0,1,0,0,0,0
39328,522630,56,0,40,2147483647,Friday,6,12,2019-04-05,4360,...,0,0,0,0,0,1,0,0,0,0
39329,522631,6,0,20,2147483647,Friday,6,12,2019-04-05,4360,...,0,0,0,0,0,1,0,0,0,0
39330,522632,18,24,6,2147483647,Friday,6,12,2019-04-05,4360,...,0,0,0,0,0,1,0,0,0,0
39331,522633,32,22,8,2147483647,Friday,6,12,2019-04-05,4360,...,0,0,0,0,0,1,0,0,0,0
39332,522634,52,16,16,2147483647,Friday,6,12,2019-04-05,4360,...,0,0,0,0,0,1,0,0,0,0
39333,522635,48,15,25,2147483647,Friday,6,12,2019-04-05,4360,...,0,0,0,0,0,1,0,0,0,0


In [73]:
mergedStuff.shape

(39550, 27)

In [74]:
mergedStuff.dtypes

ID_x                         int64
number                       int64
available_bikes              int64
available_bike_stands        int64
last_update                  int64
day_of_week_x               object
hour_d                       int64
mins                         int64
date_of_fetching            object
ID_y                         int64
temperature                float64
day_of_week_y               object
wind_speed                 float64
cloud_coverage              object
day_of_week_x_Friday         uint8
day_of_week_x_Monday         uint8
day_of_week_x_Saturday       uint8
day_of_week_x_Sunday         uint8
day_of_week_x_Thursday       uint8
day_of_week_x_Tuesday        uint8
day_of_week_x_Wednesday      uint8
cloud_coverage_Clear         uint8
cloud_coverage_Clouds        uint8
cloud_coverage_Drizzle       uint8
cloud_coverage_Fog           uint8
cloud_coverage_Mist          uint8
cloud_coverage_Rain          uint8
dtype: object

In [75]:
input_model=pd.DataFrame(mergedStuff[['number','hour_d','mins','temperature','wind_speed','day_of_week_x_Friday','day_of_week_x_Monday','day_of_week_x_Saturday','day_of_week_x_Sunday','day_of_week_x_Thursday','day_of_week_x_Tuesday','day_of_week_x_Wednesday','cloud_coverage_Clear','cloud_coverage_Clouds','cloud_coverage_Drizzle','cloud_coverage_Fog','cloud_coverage_Mist','cloud_coverage_Rain']])
#print(input_model)
output=mergedStuff['available_bikes']
X_train,X_test,Y_train,Y_test=train_test_split(input_model,output,test_size=0.33,random_state=42)
print (X_train.shape)
rf=RandomForestRegressor(n_estimators=100)
rf.fit(X_train,Y_train)
print(Y_test.shape)

(26498, 18)
(13052,)


In [76]:
#LinearReg=LinearRegression()
#LinearReg.fit(X_train,Y_train)

In [80]:
#pred=LinearReg.predict(X_test)
#pred1=pd.DataFrame(pred)
#print (pred1)
print(X_test)

#print(np.array(X_test))

       number  hour_d  mins  temperature  wind_speed  day_of_week_x_Friday  \
38870      21      14    35         5.60        10.8                     0   
2116      111       1    44         3.06         1.5                     0   
5011       61      22    17         6.13         3.1                     0   
34902      57      16    17         9.21         7.2                     0   
19850      50      20    40         5.22         7.2                     0   
9097       27      12     5         9.04         0.5                     0   
23516      81       6     2        10.22         3.6                     0   
7119       42       4    18         2.29         2.1                     0   
28788      86      23    21         6.11         1.0                     0   
25870     103      11    53         7.92         3.1                     0   
21218      35       2    27         7.20         2.6                     0   
10324      73       8    21         5.23         2.1            

In [81]:
prediction=rf.predict(X_test)
print (prediction)
print(Y_test)

[21.58 18.3   2.53 ...  8.4  28.91 12.09]
38870    15
2116     12
5011      7
34902    11
19850     4
9097     19
23516     3
7119     21
28788    38
25870     0
21218     7
10324     1
27823     4
38256     0
39535     0
21298    27
27707    13
16656    12
16169     1
19537     0
18003     0
4949      2
1736      7
4029     14
35661    14
33700    10
10062     2
32033    30
14913     2
24091     3
         ..
407       4
8538      0
1945      2
7469      1
13564    11
14881    17
23276    16
16572    20
26152     9
1357      3
13702    14
19214    25
17876    19
22567     8
16349     1
34534     0
12917     9
35252    25
1414     11
38498     0
11681     0
12166     3
29236    13
33220     5
36532    21
24117    19
35022    17
14335    10
22856    25
10761    13
Name: available_bikes, Length: 13052, dtype: int64


In [82]:
print(np.sqrt(metrics.mean_squared_error(Y_test,prediction)))

5.820983150955009


In [83]:
import pickle
pickle.dump(rf,open('final_prediction.pickle','wb'))

In [84]:
random_forest=pickle.load(open("final_prediction.pickle","rb"))

### Model to predict number of available bike stands

In [85]:

input_model_available_bike_stands=pd.DataFrame(mergedStuff[['number','hour_d','mins','temperature','wind_speed','day_of_week_x_Friday','day_of_week_x_Monday','day_of_week_x_Saturday','day_of_week_x_Sunday','day_of_week_x_Thursday','day_of_week_x_Tuesday','day_of_week_x_Wednesday','cloud_coverage_Clear','cloud_coverage_Clouds','cloud_coverage_Drizzle','cloud_coverage_Fog','cloud_coverage_Mist','cloud_coverage_Rain']])
output_bike_stands=mergedStuff['available_bike_stands']
X_train_2,X_test_2,Y_train_2,Y_test_2=train_test_split(input_model_available_bike_stands,output_bike_stands,test_size=0.33,random_state=42)
print (X_train_2.shape)
rf_2=RandomForestRegressor(n_estimators=100)
rf_2.fit(X_train_2,Y_train_2)
print(Y_test_2.shape)


(26498, 18)
(13052,)


In [86]:
prediction_bike_stands=rf_2.predict(X_test_2)
print (prediction_bike_stands)
print(Y_test_2)

[ 9.8  19.34 20.87 ... 30.05  8.63 19.2 ]
38870    15
2116     28
5011     18
34902    12
19850    36
9097      1
23516    37
7119      9
28788     0
25870    40
21218    23
10324    29
27823    16
38256    40
39535    23
21298     3
27707    17
16656    17
16169    29
19537    20
18003    40
4949     18
1736     23
4029      6
35661    26
33700    20
10062    18
32033     0
14913    38
24091    27
         ..
407      16
8538     27
1945     28
7469     29
13564    29
14881    13
23276    14
16572     0
26152    31
1357     17
13702    21
19214    15
17876    11
22567    12
16349    19
34534    40
12917    27
35252    15
1414      9
38498    30
11681    20
12166    37
29236    27
33220    25
36532     2
24117    21
35022    11
14335    30
22856    14
10761    17
Name: available_bike_stands, Length: 13052, dtype: int64


In [87]:
print(np.sqrt(metrics.mean_squared_error(Y_test_2,prediction_bike_stands)))

5.915280581118347


In [88]:
import pickle
pickle.dump(rf,open('final_prediction_bike_stands.pickle','wb'))

In [89]:
random_forest_stands=pickle.load(open("final_prediction_bike_stands.pickle","rb"))